# Mining & Modelling Character Networks

In [1]:
import pandas as pd
import networkx as nx

Paper Reference [Here](https://math.ryerson.ca/~abonato/papers/CharacterNetworks_WAW_Aug1_BDAEGH.pdf)

In [2]:
# constants
data_path = './data/data.csv'

## Mining Character Networks

## Create Network

## k-Profile

# Modelling Character Networks

## Preferential Attachement

## Chung-Lu

## Binomial Random Graph

## Configuration Model

# Machine Learning